<h1 style="text-align: center;">Some Nice Group Project Title</h1>

# Introduction

# Data Analysis

## Load Libraries

In [1]:
library(tidyverse)
library(tidymodels)
library(repr)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

## Load Dataset

In [2]:
url <-"https://github.com/mdean808/dsci-100-group-project/blob/b11c50b091b2c4a554a2b7ff8f9e568e081b0f3c/all_seasons.csv"


temp <- tempfile()


download.file(url, temp)
# read the dataset from temp file
nba_players <- read_csv("all_seasons.csv")

nba_players <- nba_players |>
filter(draft_year > 2000) |>
select(age, player_height, player_weight, pts, net_rating)

nba_players


New names:
• `` -> `...1`
Rows: 12305 Columns: 22
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (8): player_name, team_abbreviation, college, country, draft_year, draf...
dbl (14): ...1, age, player_height, player_weight, gp, pts, reb, ast, net_ra...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


age,player_height,player_weight,pts,net_rating
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
28,195.58,87.99685,5.0,6.6
23,190.50,95.25432,10.6,-6.4
32,200.66,97.52228,6.4,2.3
27,190.50,92.98636,1.0,-53.7
27,193.04,83.91452,2.4,-19.6
27,200.66,106.59412,2.6,17.5
28,177.80,72.57472,2.2,-12.4
23,210.82,129.27372,1.7,11.5
25,198.12,81.64656,0.1,-14.7


# Methods

# Expected Outcomes and Signifigance